In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
next_layer = "silver"

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
# Dynamic select query to read the data for specific proc_date
dynamic_query = f""" CREATE OR REPLACE TEMP VIEW temp_pos_wcm_saleout AS SELECT *
FROM udp_wcm_pro.udp_wcm_bronze_pro.pos_wcm_saleout
WHERE ProcDate like '{str(proc_date).replace('-', '')}%'
"""

# Execute the query
spark.sql(dynamic_query)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_payment
(
hash_id BIGINT,
  receipt_no STRING,
  pos_no STRING,
  store_id STRING,
  calday DATE,
  caltime STRING,
  line_no Double,
  ref_no STRING,
  approval_code STRING,
  bank_card_type STRING,
  bank_pos_code STRING,
  is_online Boolean,
  tender_type STRING,
  payment_amt_tendered Double,
  payment_amt_in_currency Double,
  exchange_rate Double,
  proc_date TIMESTAMP
) 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'true',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'true'
)
""")

In [0]:
spark.sql(
    f"""
CREATE OR REPLACE TEMP VIEW view_pos_wcm_saleout_payment as
WITH dat AS (
    SELECT    LTRIM('0',main_tbl.ReceiptNo) AS receipt_no,
               LTRIM('0',main_tbl.PosNo) AS pos_no,
               LTRIM('0',main_tbl.StoreCode) AS store_id, 
               to_date(to_timestamp(main_tbl.calendarday, 'MM/dd/yyyy HH:mm:ss')) AS calday,
               date_format(to_timestamp(substr(main_tbl.TranTime, 1, 6), 'HHmmss'),'HH:mm:ss') AS caltime,
               TransPaymentEntry1.LineNo AS line_no,
               TransPaymentEntry1.ReferenceNo AS ref_no,
               TransPaymentEntry1.ApprovalCode AS approval_code,
               TransPaymentEntry1.BankCardType AS bank_card_type,
               TransPaymentEntry1.BankPOSCode AS bank_pos_code,
               TransPaymentEntry1.IsOnline AS is_online,
               TransPaymentEntry1.TenderType AS tender_type,
               TransPaymentEntry1.AmountTendered AS payment_amt_tendered,
               TransPaymentEntry1.AmountInCurrency AS payment_amt_in_currency,
               TransPaymentEntry1.ExchangeRate AS exchange_rate,
               TO_TIMESTAMP(main_tbl.ProcDate, 'yyyyMMddHHmmss') AS proc_date
     FROM temp_pos_wcm_saleout main_tbl
     LATERAL VIEW explode(main_tbl.TransPaymentEntry)  AS TransPaymentEntry1 
   )
   SELECT
     farm_fingerprint(CONCAT(
         COALESCE(CAST(receipt_no AS STRING), ""),
         COALESCE(CAST(line_no AS STRING), ""),
         COALESCE(CAST(store_id AS STRING), "")
     )) AS hash_id,
           *
   FROM    dat
   QUALIFY ROW_NUMBER() OVER (PARTITION BY receipt_no, CAST(line_no AS BIGINT), store_id ORDER BY proc_date DESC) = 1
"""
)

In [0]:
spark.sql(f"DELETE FROM {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_payment WHERE RECEIPT_NO IN (SELECT RECEIPT_NO FROM view_pos_wcm_saleout_payment)")

In [0]:
spark.sql(f"INSERT INTO {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_payment SELECT * FROM view_pos_wcm_saleout_payment")